## 売上資料Ⅶ
## 月別個人別旅費内訳表　作成

- 作成年度の入力を忘れずに！

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 年度入力
nendo = 2021

# ファイル名指定
file_nm = "2021年度経費データ.csv"

In [ ]:
# 対象者指定
taisyo_nm_list = ['福','井','川','田','川','増','山','大','日','川','澤','山','関','半','村','渡','川']
keta_1_dict = {'関':'関'}
keta_3_dict = {'日':'日'}
num_3_dict = {'617':'川'}

ETC = 'ETC'
Rent = 'ﾚﾝﾀｶｰ'

In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return after

In [ ]:
# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print(list_12)

[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
target_col = ['伝票日付','借方','借方科目名','金額','消費税額','摘要']
keihidf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/" + file_nm, usecols=target_col, encoding="cp932")

In [ ]:
keihidf_i["伝票月"] = (keihidf_i['伝票日付'][:4] + keihidf_i['伝票日付'][5:-1])

In [ ]:
def date_henkan(date):
    point_cnt = 0
    henkan_list = ['00','01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(len(date)):
        if point_cnt == 2:
            mm = henkan_list[mm]
            yyyymm = int(str(yyyy) + mm)
            break

        if (point_cnt == 1) and (date[i] == "."):
            mm = int(date[first_point:i])
            point_cnt += 1

        if (point_cnt == 0) and (date[i] == "."):
            ry = int(date[1:i])
            yyyy = ry + 2018
            first_point = i + 1
            point_cnt += 1
    return yyyymm

In [ ]:
# yyyy/mm/dd -> yyyy/mm
ym_list = []
for i in range(len(keihidf_i)):
    date_keihi = date_henkan(keihidf_i.iloc[i]['伝票日付'])
    ym_list.append(date_keihi)
keihidf_i['伝票月'] = ym_list

# 金額を数字に
keihidf_i = keihidf_i.fillna(0)
kingaku_list = []
syouhizei_list = []
for i in range(len(keihidf_i)):
    kingaku_list.append(mark_num_change(str(keihidf_i.iloc[i]['金額'])))
    syouhizei_list.append(mark_num_change(str(keihidf_i.iloc[i]['消費税額'])))
keihidf_i['金額'] = kingaku_list
keihidf_i['消費税額'] = syouhizei_list

In [ ]:
keihidf_i['金額'] = keihidf_i['金額'].astype(int)
keihidf_i['消費税額'] = keihidf_i['消費税額'].astype(int)
keihidf_i['伝票月'] = keihidf_i['伝票月'].astype(int)

In [ ]:
# dict 用意
set_dict = {}
for i in list_12:
    for k in taisyo_nm_list:
        chk_nm = k
        if chk_nm[0] in keta_1_dict:
            chk_nm = keta_1_dict[chk_nm[0]]
        if chk_nm in keta_3_dict:
            chk_nm = keta_3_dict[chk_nm]
        set_dict[(i, chk_nm)] = [0, 0, 0, 0]

In [ ]:
keihidf_i = keihidf_i.sort_values(['摘要'])

In [ ]:
# 月ごとの金額セット
for i in range(len(keihidf_i)):
    if keihidf_i.iloc[i]['借方'] == 842001:
        utiwake_list = [0, 0, 0, 0]
        utiwake_list[3] = keihidf_i.iloc[i]['金額'] - keihidf_i.iloc[i]['消費税額']
        if ETC in keihidf_i.iloc[i]['摘要']:
            utiwake_list[1] = keihidf_i.iloc[i]['金額'] - keihidf_i.iloc[i]['消費税額']
        elif Rent in keihidf_i.iloc[i]['摘要']:
            utiwake_list[2] = keihidf_i.iloc[i]['金額'] - keihidf_i.iloc[i]['消費税額']
        else:
            utiwake_list[0] = keihidf_i.iloc[i]['金額'] - keihidf_i.iloc[i]['消費税額']

        chk_nm = keihidf_i.iloc[i]['摘要'][:2]
        if chk_nm[0] in keta_1_dict:
            chk_nm = keta_1_dict[chk_nm[0]]
        if chk_nm in keta_3_dict:
            chk_nm = keta_3_dict[chk_nm]
        if keihidf_i.iloc[i]['摘要'][:3] in num_3_dict:
            chk_nm = num_3_dict[keihidf_i.iloc[i]['摘要'][:3]]
        if (chk_nm in taisyo_nm_list) & (keihidf_i.iloc[i]['伝票月'] in list_12):
            set_dict[(keihidf_i.iloc[i]['伝票月'], chk_nm)][0] += utiwake_list[0]
            set_dict[(keihidf_i.iloc[i]['伝票月'], chk_nm)][1] += utiwake_list[1]
            set_dict[(keihidf_i.iloc[i]['伝票月'], chk_nm)][2] += utiwake_list[2]
            set_dict[(keihidf_i.iloc[i]['伝票月'], chk_nm)][3] += utiwake_list[3]

In [ ]:
# DataFrame作成
set_df_dict = {}
ym = list(set_dict.keys())[0][0]
value_list = []
for i, k in set_dict.items():
    if ym == i[0]:
        value_list.append(i[1])
        value_list.append(k[0])
        value_list.append(k[1])
        value_list.append(k[2])
        value_list.append(k[3])
    else:
        set_df_dict[ym] = value_list
        ym = i[0]
        value_list = []
        value_list.append(i[1])
        value_list.append(k[0])
        value_list.append(k[1])
        value_list.append(k[2])
        value_list.append(k[3])
set_df_dict[ym] = value_list

df = pd.DataFrame(set_df_dict)

In [ ]:
# Excelデータ作成
df.to_excel("/content/drive/MyDrive/Purchased_products/excel/月別個人別旅費内訳.xlsx", sheet_name='旅費')